In [9]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [11]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [12]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
print(device)

cuda


In [14]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [116]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
# Move models to device
teacher = teacher.to(device)

In [117]:

model_path = 'Best_Teacher.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [118]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
28.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

91.6%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To cha

In [18]:

# model_path = 'student_before_pruning.pth'
# # Load the model weights
# student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [119]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [120]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [121]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [122]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [123]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [124]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [125]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [126]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [127]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [128]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [129]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [130]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [131]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 2.2656 | Train Acc: 85.42% | Val Acc: 91.22%
Epoch 2/50 | Train Loss: 0.7298 | Train Acc: 94.85% | Val Acc: 91.47%
Epoch 3/50 | Train Loss: 0.4318 | Train Acc: 97.28% | Val Acc: 94.88%
Epoch 4/50 | Train Loss: 0.2911 | Train Acc: 98.62% | Val Acc: 95.49%
Epoch 5/50 | Train Loss: 0.2187 | Train Acc: 99.14% | Val Acc: 96.11%
Epoch 6/50 | Train Loss: 0.1859 | Train Acc: 99.29% | Val Acc: 96.24%
Epoch 7/50 | Train Loss: 0.1656 | Train Acc: 99.29% | Val Acc: 96.16%
Epoch 8/50 | Train Loss: 0.1524 | Train Acc: 99.28% | Val Acc: 96.33%
Epoch 9/50 | Train Loss: 0.1409 | Train Acc: 99.33% | Val Acc: 96.48%
Epoch 10/50 | Train Loss: 0.1304 | Train Acc: 99.33% | Val Acc: 96.45%
Epoch 11/50 | Train Loss: 0.1241 | Train Acc: 99.36% | Val Acc: 96.49%
Epoch 12/50 | Train Loss: 0.1165 | Train Acc: 99.34% | Val Acc: 96.41%
Epoch 13/50 | Train Loss: 0.1144 | Train Acc: 99.31% | Val Acc: 96.54%
Epoch 14/50 | Train Loss: 0.1082 | Train Acc: 99.35% | Val Acc: 96.48%
Epoch 15/50 | T

In [132]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.40%
Student Model Test Accuracy Before Pruning: 96.52%


## 93% Sparsity

In [133]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [134]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 9s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [135]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%


In [136]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 91.08%
Epoch 1/200 | Train Loss: 0.5873 | Train Acc: 82.19%
Validation Loss: 0.2535 | Validation Acc: 91.08% | Sparsity: 93.94%

New best model saved with Val Accuracy: 93.00%
Epoch 2/200 | Train Loss: 0.1390 | Train Acc: 95.37%
Validation Loss: 0.2169 | Validation Acc: 93.00% | Sparsity: 93.94%

New best model saved with Val Accuracy: 93.01%
Epoch 3/200 | Train Loss: 0.0737 | Train Acc: 97.56%
Validation Loss: 0.2182 | Validation Acc: 93.01% | Sparsity: 93.94%

New best model saved with Val Accuracy: 93.22%
Epoch 4/200 | Train Loss: 0.0379 | Train Acc: 98.79%
Validation Loss: 0.2364 | Validation Acc: 93.22% | Sparsity: 93.94%

Epoch 5/200 | Train Loss: 0.0203 | Train Acc: 99.40%
Validation Loss: 0.2598 | Validation Acc: 93.13% | Sparsity: 93.94%

New best model saved with Val Accuracy: 93.34%
Epoch 6/200 | Train Loss: 0.0124 | Train Acc: 99.67%
Validation Loss: 0.2674 | Validation Acc: 93.34% | Sparsity: 93.94%

Epoch 7/200 | Train Loss: 0.0079 

In [137]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 94.07%


In [138]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [139]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 7s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [140]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5578 | Train Acc: 92.53% | Val Loss: 0.2376 | Val Acc: 92.80% | Sparsity: 93.94%
Epoch 2/50 | Train Loss: 0.2003 | Train Acc: 97.54% | Val Loss: 0.1910 | Val Acc: 94.21% | Sparsity: 93.94%
Epoch 3/50 | Train Loss: 0.1417 | Train Acc: 98.61% | Val Loss: 0.1662 | Val Acc: 94.88% | Sparsity: 93.94%
Epoch 4/50 | Train Loss: 0.1137 | Train Acc: 99.02% | Val Loss: 0.1600 | Val Acc: 95.16% | Sparsity: 93.94%
Epoch 5/50 | Train Loss: 0.0990 | Train Acc: 99.21% | Val Loss: 0.1601 | Val Acc: 95.05% | Sparsity: 93.94%
Epoch 6/50 | Train Loss: 0.0912 | Train Acc: 99.21% | Val Loss: 0.1477 | Val Acc: 95.55% | Sparsity: 93.94%
Epoch 7/50 | Train Loss: 0.0854 | Train Acc: 99.26% | Val Loss: 0.1527 | Val Acc: 95.40% | Sparsity: 93.94%
Epoch 8/50 | Train Loss: 0.0810 | Train Acc: 99.28% | Val Loss: 0.1606 | Val Acc: 95.27% | Sparsity: 93.94%
Epoch 9/50 | Train Loss: 0.0770 | Train Acc: 99.33% | Val Loss: 0.1497 | Val Acc: 95.57% | Sparsity: 93.94%
Epoch 10/50 | Train Loss: 0.

In [141]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 95.16%


In [142]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [143]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 9s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [144]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.0157 | Train Acc: 89.17% | Val Loss: 0.4371 | Val Acc: 89.62% | Sparsity: 93.94%
Epoch 2/50 | Train Loss: 0.9076 | Train Acc: 94.91% | Val Loss: 0.3660 | Val Acc: 91.51% | Sparsity: 93.94%
Epoch 3/50 | Train Loss: 0.6276 | Train Acc: 96.77% | Val Loss: 0.2830 | Val Acc: 92.96% | Sparsity: 93.94%
Epoch 4/50 | Train Loss: 0.4558 | Train Acc: 98.04% | Val Loss: 0.2511 | Val Acc: 93.53% | Sparsity: 93.94%
Epoch 5/50 | Train Loss: 0.3555 | Train Acc: 98.60% | Val Loss: 0.1970 | Val Acc: 94.65% | Sparsity: 93.94%
Epoch 6/50 | Train Loss: 0.2883 | Train Acc: 98.95% | Val Loss: 0.1734 | Val Acc: 95.11% | Sparsity: 93.94%
Epoch 7/50 | Train Loss: 0.2542 | Train Acc: 99.04% | Val Loss: 0.1634 | Val Acc: 95.38% | Sparsity: 93.94%
Epoch 8/50 | Train Loss: 0.2328 | Train Acc: 99.09% | Val Loss: 0.1581 | Val Acc: 95.59% | Sparsity: 93.94%
Epoch 9/50 | Train Loss: 0.2160 | Train Acc: 99.14% | Val Loss: 0.1596 | Val Acc: 95.32% | Sparsity: 93.94%
Epoch 10/50 | Train Loss: 0.

In [145]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 95.38%


# 98% of Sparsity

In [146]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [147]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 9s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [148]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 54.00%
Epoch 1/200 | Train Loss: 1.9117 | Train Acc: 32.46%
Validation Loss: 1.3206 | Validation Acc: 54.00% | Sparsity: 98.44%

New best model saved with Val Accuracy: 74.27%
Epoch 2/200 | Train Loss: 0.9650 | Train Acc: 65.94%
Validation Loss: 0.7470 | Validation Acc: 74.27% | Sparsity: 98.44%

New best model saved with Val Accuracy: 78.33%
Epoch 3/200 | Train Loss: 0.6313 | Train Acc: 77.96%
Validation Loss: 0.6231 | Validation Acc: 78.33% | Sparsity: 98.44%

New best model saved with Val Accuracy: 82.85%
Epoch 4/200 | Train Loss: 0.4811 | Train Acc: 83.35%
Validation Loss: 0.5113 | Validation Acc: 82.85% | Sparsity: 98.44%

New best model saved with Val Accuracy: 83.74%
Epoch 5/200 | Train Loss: 0.3866 | Train Acc: 86.83%
Validation Loss: 0.4535 | Validation Acc: 83.74% | Sparsity: 98.44%

New best model saved with Val Accuracy: 84.89%
Epoch 6/200 | Train Loss: 0.3201 | Train Acc: 88.92%
Validation Loss: 0.4543 | Validation Acc: 84.89% | Spar

In [149]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 86.22%


In [150]:

# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


# Retrained with KD

In [151]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [152]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 96.52%


In [153]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 10s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [154]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.9340 | Train Acc: 64.36% | Val Loss: 1.1625 | Val Acc: 74.70% | Sparsity: 98.44%
Epoch 2/50 | Train Loss: 3.0879 | Train Acc: 83.14% | Val Loss: 0.7530 | Val Acc: 80.52% | Sparsity: 98.44%
Epoch 3/50 | Train Loss: 2.3303 | Train Acc: 86.98% | Val Loss: 0.5535 | Val Acc: 84.85% | Sparsity: 98.44%
Epoch 4/50 | Train Loss: 1.8804 | Train Acc: 89.46% | Val Loss: 0.6163 | Val Acc: 85.23% | Sparsity: 98.44%
Epoch 5/50 | Train Loss: 1.6104 | Train Acc: 91.00% | Val Loss: 0.6136 | Val Acc: 85.16% | Sparsity: 98.44%
Epoch 6/50 | Train Loss: 1.3999 | Train Acc: 92.15% | Val Loss: 0.6305 | Val Acc: 85.28% | Sparsity: 98.44%
Epoch 7/50 | Train Loss: 1.2028 | Train Acc: 93.61% | Val Loss: 0.5828 | Val Acc: 87.55% | Sparsity: 98.44%
Epoch 8/50 | Train Loss: 1.0590 | Train Acc: 94.33% | Val Loss: 0.4628 | Val Acc: 88.27% | Sparsity: 98.44%
Epoch 9/50 | Train Loss: 0.9546 | Train Acc: 95.03% | Val Loss: 0.4884 | Val Acc: 88.14% | Sparsity: 98.44%
Epoch 10/50 | Train Loss: 0.

In [155]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 91.57%


In [156]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [157]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 9s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [158]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.6612 | Train Acc: 65.64% | Val Loss: 0.5480 | Val Acc: 82.40% | Sparsity: 98.44%
Epoch 2/50 | Train Loss: 0.9514 | Train Acc: 86.69% | Val Loss: 0.4627 | Val Acc: 86.02% | Sparsity: 98.44%
Epoch 3/50 | Train Loss: 0.6657 | Train Acc: 90.71% | Val Loss: 0.4131 | Val Acc: 87.70% | Sparsity: 98.44%
Epoch 4/50 | Train Loss: 0.5255 | Train Acc: 92.65% | Val Loss: 0.3982 | Val Acc: 88.38% | Sparsity: 98.44%
Epoch 5/50 | Train Loss: 0.4301 | Train Acc: 94.25% | Val Loss: 0.3524 | Val Acc: 89.92% | Sparsity: 98.44%
Epoch 6/50 | Train Loss: 0.3709 | Train Acc: 95.25% | Val Loss: 0.3269 | Val Acc: 90.65% | Sparsity: 98.44%
Epoch 7/50 | Train Loss: 0.3241 | Train Acc: 95.95% | Val Loss: 0.3740 | Val Acc: 89.93% | Sparsity: 98.44%
Epoch 8/50 | Train Loss: 0.2909 | Train Acc: 96.58% | Val Loss: 0.3932 | Val Acc: 89.41% | Sparsity: 98.44%
Epoch 9/50 | Train Loss: 0.2487 | Train Acc: 97.23% | Val Loss: 0.3319 | Val Acc: 91.40% | Sparsity: 98.44%
Epoch 10/50 | Train Loss: 0.

In [159]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 92.35%


## 90% Sparsity

In [160]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [161]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 8s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [162]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 93.68%
Epoch 1/100 | Train Loss: 0.1977 | Train Acc: 94.02%
Validation Loss: 0.1848 | Validation Acc: 93.68% | Sparsity: 90.02%

New best model saved with Val Accuracy: 94.28%
Epoch 2/100 | Train Loss: 0.0566 | Train Acc: 98.17%
Validation Loss: 0.1798 | Validation Acc: 94.28% | Sparsity: 90.02%

New best model saved with Val Accuracy: 94.48%
Epoch 3/100 | Train Loss: 0.0225 | Train Acc: 99.36%
Validation Loss: 0.1974 | Validation Acc: 94.48% | Sparsity: 90.02%

New best model saved with Val Accuracy: 94.69%
Epoch 4/100 | Train Loss: 0.0093 | Train Acc: 99.79%
Validation Loss: 0.1963 | Validation Acc: 94.69% | Sparsity: 90.02%

New best model saved with Val Accuracy: 94.83%
Epoch 5/100 | Train Loss: 0.0039 | Train Acc: 99.94%
Validation Loss: 0.2049 | Validation Acc: 94.83% | Sparsity: 90.02%

New best model saved with Val Accuracy: 94.91%
Epoch 6/100 | Train Loss: 0.0023 | Train Acc: 99.97%
Validation Loss: 0.2079 | Validation Acc: 94.91% | Spar

In [163]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 94.93%


In [164]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [165]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 6s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [166]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.1616 | Train Acc: 93.50% | Val Loss: 0.3724 | Val Acc: 91.96% | Sparsity: 90.02%
Epoch 2/50 | Train Loss: 0.5320 | Train Acc: 97.41% | Val Loss: 0.2245 | Val Acc: 93.68% | Sparsity: 90.02%
Epoch 3/50 | Train Loss: 0.3489 | Train Acc: 98.59% | Val Loss: 0.1734 | Val Acc: 95.32% | Sparsity: 90.02%
Epoch 4/50 | Train Loss: 0.2644 | Train Acc: 98.97% | Val Loss: 0.1605 | Val Acc: 95.51% | Sparsity: 90.02%
Epoch 5/50 | Train Loss: 0.2258 | Train Acc: 99.07% | Val Loss: 0.1427 | Val Acc: 95.63% | Sparsity: 90.02%
Epoch 6/50 | Train Loss: 0.1988 | Train Acc: 99.19% | Val Loss: 0.1443 | Val Acc: 95.64% | Sparsity: 90.02%
Epoch 7/50 | Train Loss: 0.1842 | Train Acc: 99.12% | Val Loss: 0.1369 | Val Acc: 95.78% | Sparsity: 90.02%
Epoch 8/50 | Train Loss: 0.1726 | Train Acc: 99.14% | Val Loss: 0.1327 | Val Acc: 95.87% | Sparsity: 90.02%
Epoch 9/50 | Train Loss: 0.1608 | Train Acc: 99.18% | Val Loss: 0.1313 | Val Acc: 95.93% | Sparsity: 90.02%
Epoch 10/50 | Train Loss: 0.

In [167]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 95.67%


In [168]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [169]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 10s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [170]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.2726 | Train Acc: 96.58% | Val Loss: 0.1575 | Val Acc: 95.00% | Sparsity: 90.02%
Epoch 2/50 | Train Loss: 0.1170 | Train Acc: 98.89% | Val Loss: 0.1486 | Val Acc: 95.67% | Sparsity: 90.02%
Epoch 3/50 | Train Loss: 0.0883 | Train Acc: 99.28% | Val Loss: 0.1347 | Val Acc: 95.83% | Sparsity: 90.02%
Epoch 4/50 | Train Loss: 0.0773 | Train Acc: 99.31% | Val Loss: 0.1365 | Val Acc: 95.86% | Sparsity: 90.02%
Epoch 5/50 | Train Loss: 0.0714 | Train Acc: 99.37% | Val Loss: 0.1366 | Val Acc: 95.83% | Sparsity: 90.02%
Epoch 6/50 | Train Loss: 0.0674 | Train Acc: 99.30% | Val Loss: 0.1334 | Val Acc: 96.01% | Sparsity: 90.02%
Epoch 7/50 | Train Loss: 0.0632 | Train Acc: 99.36% | Val Loss: 0.1328 | Val Acc: 95.94% | Sparsity: 90.02%
Epoch 8/50 | Train Loss: 0.0600 | Train Acc: 99.39% | Val Loss: 0.1314 | Val Acc: 95.88% | Sparsity: 90.02%
Epoch 9/50 | Train Loss: 0.0583 | Train Acc: 99.38% | Val Loss: 0.1315 | Val Acc: 95.90% | Sparsity: 90.02%
Epoch 10/50 | Train Loss: 0.

In [171]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 95.71%


# 80% Sparsity

In [172]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [173]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 10s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [174]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 12.84%


In [175]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 95.79%
Epoch 1/100 | Train Loss: 0.0466 | Train Acc: 98.57%
Validation Loss: 0.1331 | Validation Acc: 95.79% | Sparsity: 80.02%

New best model saved with Val Accuracy: 96.12%
Epoch 2/100 | Train Loss: 0.0105 | Train Acc: 99.72%
Validation Loss: 0.1369 | Validation Acc: 96.12% | Sparsity: 80.02%

New best model saved with Val Accuracy: 96.15%
Epoch 3/100 | Train Loss: 0.0032 | Train Acc: 99.94%
Validation Loss: 0.1426 | Validation Acc: 96.15% | Sparsity: 80.02%

New best model saved with Val Accuracy: 96.22%
Epoch 4/100 | Train Loss: 0.0015 | Train Acc: 99.99%
Validation Loss: 0.1488 | Validation Acc: 96.22% | Sparsity: 80.02%

Epoch 5/100 | Train Loss: 0.0008 | Train Acc: 100.00%
Validation Loss: 0.1526 | Validation Acc: 96.18% | Sparsity: 80.02%

New best model saved with Val Accuracy: 96.35%
Epoch 6/100 | Train Loss: 0.0006 | Train Acc: 100.00%
Validation Loss: 0.1553 | Validation Acc: 96.35% | Sparsity: 80.02%

Epoch 7/100 | Train Loss: 0.000

In [176]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 96.03%


In [177]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [178]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [179]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 7s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [180]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.3333 | Train Acc: 98.43% | Val Loss: 0.1459 | Val Acc: 95.68% | Sparsity: 80.02%
Epoch 2/50 | Train Loss: 0.2014 | Train Acc: 99.08% | Val Loss: 0.1249 | Val Acc: 96.26% | Sparsity: 80.02%
Epoch 3/50 | Train Loss: 0.1578 | Train Acc: 99.14% | Val Loss: 0.1168 | Val Acc: 96.26% | Sparsity: 80.02%
Epoch 4/50 | Train Loss: 0.1377 | Train Acc: 99.22% | Val Loss: 0.1150 | Val Acc: 96.27% | Sparsity: 80.02%
Epoch 5/50 | Train Loss: 0.1253 | Train Acc: 99.22% | Val Loss: 0.1156 | Val Acc: 96.31% | Sparsity: 80.02%
Epoch 6/50 | Train Loss: 0.1169 | Train Acc: 99.17% | Val Loss: 0.1175 | Val Acc: 96.26% | Sparsity: 80.02%
Epoch 7/50 | Train Loss: 0.1119 | Train Acc: 99.18% | Val Loss: 0.1137 | Val Acc: 96.39% | Sparsity: 80.02%
Epoch 8/50 | Train Loss: 0.1032 | Train Acc: 99.17% | Val Loss: 0.1100 | Val Acc: 96.52% | Sparsity: 80.02%
Epoch 9/50 | Train Loss: 0.0988 | Train Acc: 99.20% | Val Loss: 0.1109 | Val Acc: 96.51% | Sparsity: 80.02%
Epoch 10/50 | Train Loss: 0.

In [181]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 96.12%


In [182]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [183]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [184]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.0948 | Train Acc: 98.97% | Val Loss: 0.1190 | Val Acc: 96.04% | Sparsity: 80.02%
Epoch 2/50 | Train Loss: 0.0580 | Train Acc: 99.40% | Val Loss: 0.1113 | Val Acc: 96.46% | Sparsity: 80.02%
Epoch 3/50 | Train Loss: 0.0511 | Train Acc: 99.43% | Val Loss: 0.1103 | Val Acc: 96.46% | Sparsity: 80.02%
Epoch 4/50 | Train Loss: 0.0466 | Train Acc: 99.41% | Val Loss: 0.1089 | Val Acc: 96.56% | Sparsity: 80.02%
Epoch 5/50 | Train Loss: 0.0433 | Train Acc: 99.42% | Val Loss: 0.1114 | Val Acc: 96.46% | Sparsity: 80.02%
Epoch 6/50 | Train Loss: 0.0417 | Train Acc: 99.44% | Val Loss: 0.1097 | Val Acc: 96.49% | Sparsity: 80.02%
Epoch 7/50 | Train Loss: 0.0406 | Train Acc: 99.45% | Val Loss: 0.1113 | Val Acc: 96.51% | Sparsity: 80.02%
Epoch 8/50 | Train Loss: 0.0392 | Train Acc: 99.42% | Val Loss: 0.1094 | Val Acc: 96.53% | Sparsity: 80.02%
Epoch 9/50 | Train Loss: 0.0380 | Train Acc: 99.45% | Val Loss: 0.1104 | Val Acc: 96.48% | Sparsity: 80.02%
Early stopping triggered at 

In [185]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 96.08%


# 75% of Sparsity

In [186]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [187]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 9s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [188]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.17%


In [189]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 96.15%
Epoch 1/100 | Train Loss: 0.0305 | Train Acc: 99.06%
Validation Loss: 0.1224 | Validation Acc: 96.15% | Sparsity: 75.01%

New best model saved with Val Accuracy: 96.45%
Epoch 2/100 | Train Loss: 0.0067 | Train Acc: 99.81%
Validation Loss: 0.1289 | Validation Acc: 96.45% | Sparsity: 75.01%

New best model saved with Val Accuracy: 96.51%
Epoch 3/100 | Train Loss: 0.0022 | Train Acc: 99.97%
Validation Loss: 0.1352 | Validation Acc: 96.51% | Sparsity: 75.01%

Epoch 4/100 | Train Loss: 0.0011 | Train Acc: 99.99%
Validation Loss: 0.1427 | Validation Acc: 96.48% | Sparsity: 75.01%

New best model saved with Val Accuracy: 96.54%
Epoch 5/100 | Train Loss: 0.0007 | Train Acc: 100.00%
Validation Loss: 0.1419 | Validation Acc: 96.54% | Sparsity: 75.01%

Epoch 6/100 | Train Loss: 0.0004 | Train Acc: 100.00%
Validation Loss: 0.1436 | Validation Acc: 96.48% | Sparsity: 75.01%

New best model saved with Val Accuracy: 96.59%
Epoch 7/100 | Train Loss: 0.000

In [190]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.16%


In [191]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

# Retrain with KD

In [192]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [193]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 7s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [194]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.2018 | Train Acc: 99.11% | Val Loss: 0.1261 | Val Acc: 96.14% | Sparsity: 75.01%
Epoch 2/50 | Train Loss: 0.1470 | Train Acc: 99.08% | Val Loss: 0.1123 | Val Acc: 96.48% | Sparsity: 75.01%
Epoch 3/50 | Train Loss: 0.1274 | Train Acc: 99.17% | Val Loss: 0.1108 | Val Acc: 96.45% | Sparsity: 75.01%
Epoch 4/50 | Train Loss: 0.1130 | Train Acc: 99.16% | Val Loss: 0.1067 | Val Acc: 96.58% | Sparsity: 75.01%
Epoch 5/50 | Train Loss: 0.1037 | Train Acc: 99.26% | Val Loss: 0.1079 | Val Acc: 96.59% | Sparsity: 75.01%
Epoch 6/50 | Train Loss: 0.0989 | Train Acc: 99.22% | Val Loss: 0.1082 | Val Acc: 96.60% | Sparsity: 75.01%
Epoch 7/50 | Train Loss: 0.0926 | Train Acc: 99.20% | Val Loss: 0.1078 | Val Acc: 96.55% | Sparsity: 75.01%
Epoch 8/50 | Train Loss: 0.0882 | Train Acc: 99.20% | Val Loss: 0.1053 | Val Acc: 96.47% | Sparsity: 75.01%
Epoch 9/50 | Train Loss: 0.0847 | Train Acc: 99.22% | Val Loss: 0.1069 | Val Acc: 96.52% | Sparsity: 75.01%
Epoch 10/50 | Train Loss: 0.

In [195]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.14%


In [196]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

In [197]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [198]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [199]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.0686 | Train Acc: 99.21% | Val Loss: 0.1107 | Val Acc: 96.50% | Sparsity: 75.01%
Epoch 2/50 | Train Loss: 0.0462 | Train Acc: 99.42% | Val Loss: 0.1070 | Val Acc: 96.60% | Sparsity: 75.01%
Epoch 3/50 | Train Loss: 0.0422 | Train Acc: 99.45% | Val Loss: 0.1076 | Val Acc: 96.54% | Sparsity: 75.01%
Epoch 4/50 | Train Loss: 0.0397 | Train Acc: 99.44% | Val Loss: 0.1082 | Val Acc: 96.60% | Sparsity: 75.01%
Epoch 5/50 | Train Loss: 0.0378 | Train Acc: 99.42% | Val Loss: 0.1067 | Val Acc: 96.62% | Sparsity: 75.01%
Epoch 6/50 | Train Loss: 0.0359 | Train Acc: 99.43% | Val Loss: 0.1057 | Val Acc: 96.72% | Sparsity: 75.01%
Epoch 7/50 | Train Loss: 0.0344 | Train Acc: 99.47% | Val Loss: 0.1067 | Val Acc: 96.61% | Sparsity: 75.01%
Epoch 8/50 | Train Loss: 0.0340 | Train Acc: 99.47% | Val Loss: 0.1084 | Val Acc: 96.78% | Sparsity: 75.01%
Epoch 9/50 | Train Loss: 0.0325 | Train Acc: 99.50% | Val Loss: 0.1087 | Val Acc: 96.76% | Sparsity: 75.01%
Epoch 10/50 | Train Loss: 0.

In [200]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.13%


## 50% Sparsity

In [201]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [202]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [203]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 73.05%


In [204]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 96.82%
Epoch 1/200 | Train Loss: 0.0158 | Train Acc: 99.42%
Validation Loss: 0.1060 | Validation Acc: 96.82% | Sparsity: 50.47%

New best model saved with Val Accuracy: 96.94%
Epoch 2/200 | Train Loss: 0.0036 | Train Acc: 99.89%
Validation Loss: 0.1140 | Validation Acc: 96.94% | Sparsity: 50.47%

Epoch 3/200 | Train Loss: 0.0012 | Train Acc: 99.99%
Validation Loss: 0.1210 | Validation Acc: 96.87% | Sparsity: 50.47%

Epoch 4/200 | Train Loss: 0.0005 | Train Acc: 100.00%
Validation Loss: 0.1241 | Validation Acc: 96.89% | Sparsity: 50.47%

New best model saved with Val Accuracy: 96.97%
Epoch 5/200 | Train Loss: 0.0004 | Train Acc: 100.00%
Validation Loss: 0.1288 | Validation Acc: 96.97% | Sparsity: 50.47%

Epoch 6/200 | Train Loss: 0.0003 | Train Acc: 100.00%
Validation Loss: 0.1300 | Validation Acc: 96.97% | Sparsity: 50.47%

New best model saved with Val Accuracy: 97.00%
Epoch 7/200 | Train Loss: 0.0002 | Train Acc: 100.00%
Validation Loss: 0.1310

In [205]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.52%


In [206]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [207]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [208]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.0847 | Train Acc: 99.43% | Val Loss: 0.1033 | Val Acc: 96.70% | Sparsity: 50.47%
Epoch 2/50 | Train Loss: 0.0778 | Train Acc: 99.28% | Val Loss: 0.1017 | Val Acc: 96.70% | Sparsity: 50.47%
Epoch 3/50 | Train Loss: 0.0751 | Train Acc: 99.24% | Val Loss: 0.1019 | Val Acc: 96.66% | Sparsity: 50.47%
Epoch 4/50 | Train Loss: 0.0709 | Train Acc: 99.24% | Val Loss: 0.1019 | Val Acc: 96.72% | Sparsity: 50.47%
Epoch 5/50 | Train Loss: 0.0680 | Train Acc: 99.24% | Val Loss: 0.1011 | Val Acc: 96.73% | Sparsity: 50.47%
Epoch 6/50 | Train Loss: 0.0673 | Train Acc: 99.24% | Val Loss: 0.1021 | Val Acc: 96.69% | Sparsity: 50.47%
Epoch 7/50 | Train Loss: 0.0636 | Train Acc: 99.25% | Val Loss: 0.1024 | Val Acc: 96.70% | Sparsity: 50.47%
Epoch 8/50 | Train Loss: 0.0612 | Train Acc: 99.29% | Val Loss: 0.1001 | Val Acc: 96.78% | Sparsity: 50.47%
Epoch 9/50 | Train Loss: 0.0605 | Train Acc: 99.26% | Val Loss: 0.1025 | Val Acc: 96.80% | Sparsity: 50.47%
Epoch 10/50 | Train Loss: 0.

In [209]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.49%


In [210]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [211]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 4s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [212]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.0314 | Train Acc: 99.44% | Val Loss: 0.0992 | Val Acc: 96.91% | Sparsity: 50.47%
Epoch 2/50 | Train Loss: 0.0275 | Train Acc: 99.48% | Val Loss: 0.0994 | Val Acc: 96.91% | Sparsity: 50.47%
Epoch 3/50 | Train Loss: 0.0263 | Train Acc: 99.47% | Val Loss: 0.0996 | Val Acc: 96.92% | Sparsity: 50.47%
Epoch 4/50 | Train Loss: 0.0254 | Train Acc: 99.50% | Val Loss: 0.1016 | Val Acc: 96.99% | Sparsity: 50.47%
Epoch 5/50 | Train Loss: 0.0248 | Train Acc: 99.50% | Val Loss: 0.1014 | Val Acc: 96.97% | Sparsity: 50.47%
Epoch 6/50 | Train Loss: 0.0238 | Train Acc: 99.51% | Val Loss: 0.1020 | Val Acc: 96.91% | Sparsity: 50.47%
Epoch 7/50 | Train Loss: 0.0238 | Train Acc: 99.49% | Val Loss: 0.1012 | Val Acc: 96.90% | Sparsity: 50.47%
Epoch 8/50 | Train Loss: 0.0233 | Train Acc: 99.52% | Val Loss: 0.1034 | Val Acc: 96.88% | Sparsity: 50.47%
Epoch 9/50 | Train Loss: 0.0233 | Train Acc: 99.47% | Val Loss: 0.1023 | Val Acc: 96.98% | Sparsity: 50.47%
Early stopping triggered at 

In [213]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.51%
